## Matlab MIGP

This notebook will load the dimension reduced data from Matlab MIGP, run group ICA, and then plot the group ICs.

* [Run `melodic`](#run-matlab-melodic)
* [Plot group ICs](#plot-matlab-group-ics)

Firstly we will import the necessary packages for this notebook:  

In [ ]:
from nilearn import plotting
from nilearn import image
import nibabel as nb
import matplotlib.pyplot as plt
import numpy as np
import os.path as op

It will be necessary to know the location where the data was stored so that we can load the brainmask:

> **Note**: [`expanduser`](https://docs.python.org/3.7/library/os.path.html#os.path.expanduser) will expand the `~` to the be users home directory

In [ ]:
data_dir = op.expanduser('~/nilearn_data')

<a class="anchor" id="run-matlab-melodic"></a>
### Run ```melodic```

Generate a command line string and run group ```melodic``` on the Matlab MIGP dimension reduced data with a dimension of 10 components.  We disable MIGP because it was already run separately in Matlab.

> **Note**: 
> 1. Here we use python [f-strings](https://www.python.org/dev/peps/pep-0498/), formally known as literal string interpolation, which allow for easy formatting
> 2. [`op.join`](https://docs.python.org/3.7/library/os.path.html#os.path.join) will join path strings using the platform-specific directory separator

In [ ]:
# generate melodic command line string
melodic_cmd = f"melodic -i matMIGP.nii.gz --mask={op.join(data_dir, 'brain_mask.nii.gz')} -d 10 -v --nobet --disableMigp -o matmigp.gica"
print(melodic_cmd)

> **Note:** 
> 1. Here we use the `!` operator to execute the command in the shell
> 2. The `{}` will expand the contained python variable in the shell

In [ ]:
# run melodic
! {melodic_cmd}

<a class="anchor" id="plot-matlab-group-ics"></a>
### Plot group ICs

Now we can load and plot the group ICs generated by ```melodic```.

This function will be used to plot ICs:

> **NOTE:**
> 1. Here we use [`plot_stat_map`](https://nilearn.github.io/modules/generated/nilearn.plotting.plot_stat_map.html) from the [`nilearn`](https://nilearn.github.io/index.html) package to plot the orthographic images
> 2. [`subplots`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplots.html) from `matplotlib.pyplot` creates a figure and multiple subplots
> 3. [`find_xyz_cut_coords`](https://nilearn.github.io/modules/generated/nilearn.plotting.find_xyz_cut_coords.html) from the [`nilearn`](https://nilearn.github.io/index.html) package will find the image coordinates of the center of the largest activation connected component
> 4. [`zip`](https://docs.python.org/3.3/library/functions.html#zip) takes iterables and aggregates them in a tuple.  Here it is used to iterate through two lists simultaneously
> 5. [`iter_img`](https://nilearn.github.io/modules/generated/nilearn.image.iter_img.html) from the [`nilearn`](https://nilearn.github.io/index.html) package creates an iterator from an image that steps through each volume/time-point of the image

In [ ]:
def map_plot(d):

    N = d.shape[-1]

    fig, ax = plt.subplots(int(np.ceil((N/2))),2, figsize=(12, N))

    for img, ax0 in zip(image.iter_img(d), ax.ravel()):
        coord = plotting.find_xyz_cut_coords(img, activation_threshold=3.5)
        plotting.plot_stat_map(img, cut_coords=coord, vmax=10, axes=ax0)
        
    return fig

Hopefully you can see some familiar looking RSN spatial patterns:

In [ ]:
ics = nb.load('matmigp.gica/melodic_IC.nii.gz')
fig = map_plot(ics)